In [2]:
import os

In [3]:
!pwd

/Users/sanjaymahto/Chess-Cancer-Classification/Chest-Cancer-Project/research


In [4]:
os.chdir("../")

In [5]:
# from pathlib import Path
# cwd = Path.cwd().parent.parent
# print(cwd)

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) # Because of frozen=True, we can't change the values of the fields after the object is created
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.CNN.constants import *
from src.CNN.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    
    

In [8]:
import os 
import zipfile
import gdown 
from CNN import logger
from CNN.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self):
        """Fetch data from the url 
        """
        
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            # logger.error(f"Error downloading data from {dataset_url} into file {zip_download_dir}")
            # logger.error(str(e))
            raise e
            
    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir # This will create directory path if it does not exist. 
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [10]:
# upgrade the pipeline
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    # logger.error(f"Error in data ingestion step: {str(e)}")
    raise e

[2024-10-25 22:58:13,679 :INFO: common: Reading yaml file: config/config.yml loaded successfully]
[2024-10-25 22:58:13,682 :INFO: common: Reading yaml file: params.yaml loaded successfully]
[2024-10-25 22:58:13,682 :INFO: common: created directory at: artifacts]
[2024-10-25 22:58:13,683 :INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-25 22:58:13,683 :INFO: 2887685291: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.csv]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=6870ea42-7efc-4624-aeec-25510f2e1bc5
To: /Users/sanjaymahto/Chess-Cancer-Classification/Chest-Cancer-Project/artifacts/data_ingestion/data.csv
100%|██████████| 49.0M/49.0M [00:17<00:00, 2.87MB/s]

[2024-10-25 22:58:36,659 :INFO: 2887685291: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.csv]
